In [1]:
import os
import time
import sys
import re

import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains

In [2]:
PATH = 'C:/21_hf216/project/CrawlingData/'
df = pd.read_csv(PATH + 'book_info.csv')

print(df.shape)
df.head()

(5906, 4)


,0,1,2,3
0,9771197225987,흘러가는 구름을 동경하였다,"김시탁, 김일태, 민창홍, 성선경, 이강휘, 이기영, 이달균, 이서린, 이월춘",시문학연구회 「하로동선」의 여섯 번째 동인지. 지금까지 현대시의 새로운 방향성을 탐...
1,9772586200006,장애와 테크놀로지 - 과학잡지 에피 16호,"김초엽, 김원영, 김상희, 유기훈, 이주현, 류성두, 명수민, 킴 포춘, 전치형, ...",『에피』 16호 키워드 숨 섹션은 ‘장애와 테크놀로지’를 주제로 삼았다. 기술의 발...
2,9772586200211,식물의 과학 - 과학잡지 에피 17호,"강연실, 허준이, 금종해, 안수지, 이상엽, 남재환, 황승식, 장진성, 박찬열, 강...",과학비평 계간지 에피 17호는 “식물의 과학”을 다루며 17편의 글을 담았다. 식물...
3,9772733807003,"에픽 #04 - BELOVED, 2021.7.8.9","임현, 임철홍, 에밀리 정민 윤, 이정식, 임소라, 김대성, 박하빈, 염은영, 김혜...",4호의 제호는 ‘BELOVED’이다. 1988년 퓰리처상 수상작이기도 한 토니 모리...
4,9772765656006,오늘의 좋은 소설 2021.가을 - 58호,"김현, 서정아, 이미욱, 이태형, 정미형",오늘의 좋은 소설은 간단하다. 무겁지도 어렵지도 않다. 어떤 장식과 해설도 하지 않...


In [3]:
isbn_list = df.iloc[:,0]
isbn_list = isbn_list.apply(str)
isbn_list

0       9771197225987
1       9772586200006
2       9772586200211
3       9772733807003
4       9772765656006
            ...      
5901    9791197556609
5902    9791197558009
5903    9791197559105
5904    9791197566400
5905    9791197578410
Name: 0, Length: 5906, dtype: object

In [4]:
# 검색창에 isbn 입력하는 함수
def close_popup():
    css = '#divYes24SCMEvent > div.yPopBot'
    popup = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css)))
    btnClick1 = ActionChains(driver).move_to_element(popup).click()
    btnClick1.perform()
    print('팝업 닫기')
    time.sleep(1)

def search_isbn(isbn):
    css = '#query'
    search_box = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    # search_box = driver.find_element_by_css_selector(class_)
    search_box.send_keys(isbn)
    search_box.submit()
    if search_box:
        print(f'{isbn} 검색 완료')
    
    time.sleep(1)

# 검색 후 작가 클릭 하는 함수
def click_author():
    driver.refresh()
    css = '#schMid_wrap > div.goods_list_wrap.mgt30 > div.goodsList.goodsList_list > table > tbody > tr:nth-child(1) > td.goods_infogrp > div.goods_info > a:nth-child(1)'
    # author = driver.find_element_by_css_selector(css)
    author = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css)))
    btnClick = ActionChains(driver).move_to_element(author).click()
    btnClick.perform()
    print('작가 버튼 클릭')

def click_more():
    driver.refresh()
    css_more = '#schMid_wrap > div.insideYes24.insideTp01.clearfix > div > dl > dd > a'
    more = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css_more)))
    btnClick2 = ActionChains(driver).move_to_element(more).click()
    btnClick2.perform()
    print('작가 더보기 버튼 클릭')

def get_info_text():
    driver.refresh()
    info_text = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, 'txt150'))).text
    # info_text = driver.find_element_by_class_name('txt150').text
    print('작가 정보 저장')
    return info_text


In [7]:
from tqdm import tqdm
author_infolist = []
error_list = []
chromedriver = 'C:/chromedriver'
driver = webdriver.Chrome(chromedriver)
driver.get('http://www.yes24.com/main/default.aspx')
close_popup()
for isbn in tqdm(isbn_list):
    driver.get('http://www.yes24.com/main/default.aspx')
    try:
        search_isbn(isbn)
        click_author()
        click_more()
        author_info = get_info_text()
        author_infolist.append([isbn, author_info])
    except Exception as e:
        print(e)
        author_infolist.append([np.nan])
        author_info = np.nan
        error_list.append([isbn, e])
    # with open('author_info2.csv', encoding='utf-8-sig', mode='a') as f:
    #     f.write(f'{author_info}\n')

In [113]:
author_series = pd.Series()
for i in range(len(author_infolist)):
    author_series = author_series.append(pd.Series(author_infolist[i][0]))
author_series.reset_index(drop=True, inplace=True)
author_series

C:\ProgramData\Anaconda3\envs\tens_2g\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


0                                                    NaN
1                                                    NaN
2                                                    NaN
3      1983년 전남 순천에서 태어났다. 2014년 [현대문학] 신인추천에 단편소설 『그...
4      1817년 매사추세츠 주 콩코드에서 태어났다. 자신을 ‘신비주의자, 초절주의자, 자...
                             ...                        
765    어린이청소년문학 작가. 1962년 충북 청원군에서 나고 서울에서 자랐다. 유년기부터...
766    여러 도시와 해외를 전전하다 포항에서 도서출판 이팝을 운영 중이다. 생계형 기자였다...
767    진주에서 태어나 서울에서 자랐다. 어렸을 때, 골방에서 홀로 책을 읽던 시간의 후유...
768    1957년 도쿄에서 태어났다. 국제상과대학 상학부를 졸업한 뒤 조모신문(上毛新聞)에...
769    1985년 경기도 성남에서 태어나 부산에서 어린 시절을 보냈다. 주민등록증이 나왔을...
Length: 770, dtype: object

In [124]:
sub = pd.DataFrame(dict(isbn13 = isbn_list, name = author_series)).reset_index(drop=True)
sub['name'].fillna('작가 정보가 없습니다.', inplace=True)
sub.to_csv('author_info.csv', encoding='utf-8-sig', index=False)
sub

,isbn13,name
0,9772384289005,작가 정보가 없습니다.
1,9772384367000,작가 정보가 없습니다.
2,9772508333003,작가 정보가 없습니다.
3,9772733807003,1983년 전남 순천에서 태어났다. 2014년 [현대문학] 신인추천에 단편소설 『그...
4,9788901166223,"1817년 매사추세츠 주 콩코드에서 태어났다. 자신을 ‘신비주의자, 초절주의자, 자..."
...,...,...
765,9791197120541,어린이청소년문학 작가. 1962년 충북 청원군에서 나고 서울에서 자랐다. 유년기부터...
766,9791197182204,여러 도시와 해외를 전전하다 포항에서 도서출판 이팝을 운영 중이다. 생계형 기자였다...
767,9791197381706,"진주에서 태어나 서울에서 자랐다. 어렸을 때, 골방에서 홀로 책을 읽던 시간의 후유..."
768,9791197459702,1957년 도쿄에서 태어났다. 국제상과대학 상학부를 졸업한 뒤 조모신문(上毛新聞)에...


In [22]:
author_info = pd.read_csv(PATH + 'author_info.csv')
print(author_info.shape)
author_info.head()

(770, 2)


,isbn13,name
0,9772384289005,작가 정보가 없습니다.
1,9772384367000,작가 정보가 없습니다.
2,9772508333003,작가 정보가 없습니다.
3,9772733807003,1983년 전남 순천에서 태어났다. 2014년 [현대문학] 신인추천에 단편소설 『그...
4,9788901166223,"1817년 매사추세츠 주 콩코드에서 태어났다. 자신을 ‘신비주의자, 초절주의자, 자..."


In [5]:
book_info = pd.read_csv(PATH + 'book_info.csv')
print(book_info.shape)
# 컬럼명 변경
book_info.rename(columns={'0': 'isbn13', '1': 'title', '2': 'author'}, inplace=True)
book_info.head()

(770, 3)


,isbn13,title,author
0,9772384289005,미스테리아 35호,미스테리아 편집부
1,9772384367000,악스트 Axt 2021.7.8 - no.037,악스트 편집부 (지은이)
2,9772508333003,릿터 Littor 2021.6.7 - 30호,릿터 편집부 (지은이)
3,9772733807003,"에픽 #04 - BELOVED, 2021.7.8.9","임현, 임철홍, 에밀리 정민 윤, 이정식, 임소라, 김대성, 박하빈, 염은영, 김혜..."
4,9788901166223,월든 - 시민 불복종 수록,"헨리 데이비드 소로우 (지은이), 홍지수 (옮긴이)"


In [24]:
author_info.head(3)

,isbn13,name
0,9772384289005,작가 정보가 없습니다.
1,9772384367000,작가 정보가 없습니다.
2,9772508333003,작가 정보가 없습니다.


In [25]:
merge1.head(3)

,idx,isbn13,title,author
0,1,9772384289005,미스테리아 35호,미스테리아 편집부
1,2,9772384367000,악스트 Axt 2021.7.8 - no.037,악스트 편집부 (지은이)
2,3,9772508333003,릿터 Littor 2021.6.7 - 30호,릿터 편집부 (지은이)


In [6]:
from tqdm import tqdm
list_ = []
for i in range(len(book_info)):
    list_.append(i+1)
idx = pd.DataFrame(dict(idx = list_, isbn13 = book_info['isbn13'])).reset_index(drop=True)
merge1 = pd.merge(idx, book_info, on='isbn13', how='left')
merge2 = pd.merge(merge1, author_info, how='left', on='isbn13')
merge2.rename(columns={'name': 'description'}, inplace=True)
merge2[['booklist', 'cleaned', 'vector']] = np.nan

li_ = []
for i in tqdm(range(len(merge2))):
    if '(' in merge2.loc[i, 'author']:
        merge2.loc[i, 'author'] = merge2.loc[i, 'author'].split('(')[0].strip()
    if merge2.loc[i, 'description'] == '작가 정보가 없습니다.':
        merge2.loc[i, 'description'] = np.nan
    if len(merge2.loc[i, 'author'].split(',')) > 1:
        li_.append(i)
merge2.iloc[li_, :]
merge2.to_csv(PATH + 'merge2_removed.csv', encoding='utf-8-sig', index=False)

NameError: name 'author_info' is not defined

In [1]:

merge2.to_csv(PATH + 'author_table.csv', encoding='utf-8-sig', index=False)
merge2.head(3)

NameError: name 'merge2' is not defined